# EDA

## Importar Librerias

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy.stats as stats 

## Preparacion del dataframe para su analisis

In [2]:
steam = pd.read_json('../Datasets/Steam_Games_Limpio.json.gz', compression='gzip')
review = pd.read_json('../Datasets/User_Reviews_Limpio.json.gz', compression='gzip')
items = pq.read_table('Datasets OPS\items.parquet').to_pandas()

In [3]:
# Creamos una linea por item del usuario
items = items.explode('items', ignore_index=True)
itemsDeUsuario = pd.json_normalize(items['items'])
items.head()

,user_id,items_count,items
0,76561197970982479,277,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [4]:
itemsDeUsuario

,item_id,item_name,playtime_2weeks,playtime_forever
0,10,Counter-Strike,0.0,6.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,0.0,7.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0
5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,519140,Minds Eyes,3.0,3.0


In [5]:
# borramos las filas donde tienen todos nulos, borramos playtime_2weeks ya que no me da informacion.
itemsDeUsuario.dropna(how='all', inplace=True)
itemsDeUsuario['item_id'] = itemsDeUsuario['item_id'].astype(int)
itemsDeUsuario.drop(columns='playtime_2weeks', inplace=True)

# Preparamos items para hacer un merge con los items ya en forma de dataframe.
items.dropna(subset='items', inplace=True)
items.drop(columns='items',inplace=True)

In [7]:
itemsDeUsuario

,item_id,item_name,playtime_forever
0,10,Counter-Strike,6.0
1,20,Team Fortress Classic,0.0
2,30,Day of Defeat,7.0
3,40,Deathmatch Classic,0.0
4,50,Half-Life: Opposing Force,0.0
...,...,...,...
5170009,346330,BrainBread 2,0.0
5170010,373330,All Is Dust,0.0
5170011,388490,One Way To Die: Steam Edition,3.0
5170012,521570,You Have 10 Seconds 2,4.0


In [6]:
review

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,1
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2
...,...,...,...,...
59328,76561198312638244,70,True,2
59329,76561198312638244,362890,True,2
59330,LydiaMorley,273110,True,2
59331,LydiaMorley,730,True,2


In [9]:
# Generamos un dataframe con toda la informacion junta, la de reviews al hacer el merge y la de items.
df = review.merge(items, on='user_id', how='left').drop_duplicates()
df.reset_index(drop=True, inplace=True)

items = pd.concat([items, itemsDeUsuario], axis=1)
df = df.merge(items.drop(columns='items_count'), on=['user_id','item_id'])

In [10]:
df

,user_id,item_id,recommend,sentiment_analysis,items_count,item_name,playtime_forever
0,76561197970982479,1250,True,2,277.0,Killing Floor,10006.0
1,76561197970982479,22200,True,1,277.0,Zeno Clash,271.0
2,76561197970982479,43110,True,2,277.0,Metro 2033,834.0
3,js41637,251610,True,2,888.0,Barbie‚Ñ¢ Dreamhouse Party‚Ñ¢,84.0
4,js41637,227300,True,2,888.0,Euro Truck Simulator 2,551.0
...,...,...,...,...,...,...,...
44888,Ghoustik,730,True,1,4.0,Counter-Strike: Global Offensive,3969.0
44889,76561198312638244,233270,True,2,36.0,Far Cry¬Æ 3 Blood Dragon,533.0
44890,76561198312638244,130,True,2,36.0,Half-Life: Blue Shift,221.0
44891,76561198312638244,70,True,2,36.0,Half-Life,1010.0


In [11]:
steam

,genres,title,release_date,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,0.00,643980,Secret Level Srl
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,0.00,670290,Poolians.Com
3,"[Action, Adventure, Casual]",弹炸人2222,2017,0.99,767400,彼岸领域
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...,...
28661,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018,1.99,745400,Bidoniera Games
28662,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018,1.99,773640,"Nikita ""Ghost_Rus"""
28663,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018,4.99,733530,Sacada
28664,"[Indie, Racing, Simulation]",Russian Roads,2018,1.99,610660,Laush Dmitriy Sergeevich


In [12]:
# Ya unidos los dataframes de items, y review, ahora juntamos con el de steam para tener toda la informacion junta para analisarla.
df = df.merge(steam, left_on='item_id', right_on='id')
df = df.drop(columns=['id','title'])
df[['playtime_forever','items_count']]=df[['playtime_forever','items_count']].astype(int)

In [13]:
df

,user_id,item_id,recommend,sentiment_analysis,items_count,item_name,playtime_forever,genres,release_date,price,developer
0,76561197970982479,1250,True,2,277,Killing Floor,10006,[Action],2009,19.99,Tripwire Interactive
1,76561197970982479,22200,True,1,277,Zeno Clash,271,"[Action, Indie]",2009,9.99,Ace Team
2,js41637,227300,True,2,888,Euro Truck Simulator 2,551,"[Indie, Simulation]",2013,19.99,Scs Software
3,js41637,239030,True,2,888,"Papers, Please",349,"[Adventure, Indie]",2013,9.99,3909
4,evcentric,248820,True,1,137,Risk of Rain,2328,"[Action, Indie, RPG]",2013,9.99,"Hopoo Games, Llc"
...,...,...,...,...,...,...,...,...,...,...,...
39009,Ghoustik,730,True,1,4,Counter-Strike: Global Offensive,3969,[Action],2012,14.99,Valve
39010,76561198312638244,233270,True,2,36,Far Cry¬Æ 3 Blood Dragon,533,"[Action, Adventure]",2013,14.99,Ubisoft Montreal
39011,76561198312638244,130,True,2,36,Half-Life: Blue Shift,221,[Action],2001,4.99,Gearbox Software
39012,76561198312638244,70,True,2,36,Half-Life,1010,[Action],1998,9.99,Valve


In [15]:
# Cambiamos el tipo de recommend de bool a int
df['recommend'] = df['recommend'].astype('int8')
df

,user_id,item_id,recommend,sentiment_analysis,items_count,item_name,playtime_forever,genres,release_date,price,developer
0,76561197970982479,1250,1,2,277,Killing Floor,10006,[Action],2009,19.99,Tripwire Interactive
1,76561197970982479,22200,1,1,277,Zeno Clash,271,"[Action, Indie]",2009,9.99,Ace Team
2,js41637,227300,1,2,888,Euro Truck Simulator 2,551,"[Indie, Simulation]",2013,19.99,Scs Software
3,js41637,239030,1,2,888,"Papers, Please",349,"[Adventure, Indie]",2013,9.99,3909
4,evcentric,248820,1,1,137,Risk of Rain,2328,"[Action, Indie, RPG]",2013,9.99,"Hopoo Games, Llc"
...,...,...,...,...,...,...,...,...,...,...,...
39009,Ghoustik,730,1,1,4,Counter-Strike: Global Offensive,3969,[Action],2012,14.99,Valve
39010,76561198312638244,233270,1,2,36,Far Cry¬Æ 3 Blood Dragon,533,"[Action, Adventure]",2013,14.99,Ubisoft Montreal
39011,76561198312638244,130,1,2,36,Half-Life: Blue Shift,221,[Action],2001,4.99,Gearbox Software
39012,76561198312638244,70,1,2,36,Half-Life,1010,[Action],1998,9.99,Valve


In [ ]:
# Expotrtamos para su uso con el modelo de machine learning
df.to_json('EDA\EDA_Dataset.json.gz', compression='gzip')

## Comienzo del analisis

### ***Top 5 juego con mas horas jugadas (playtime_forever)***

In [37]:
# Sacamos las horas del dataframe
juegoMasJugado = df[['item_name', 'playtime_forever']].groupby('item_name', as_index=False).sum()
# Agarramos el top 5
juegoMasJugado = juegoMasJugado.sort_values('playtime_forever', ascending=False).head(5)


fig = px.bar(
        juegoMasJugado,
        x='item_name', 
        y='playtime_forever',
        title='<b>Top 5 juegos con mas horas<b>',
        labels={'item_name': 'Nombre del juego', 'playtime_forever': 'Horas jugadas'},
        text_auto=True,
        width=800,
        color_discrete_sequence=['#17becf']
            )
fig.show()

### ***Analisis de price***

In [38]:
# Crear subplots (1 fila, 2 columnas)
fig = make_subplots(rows=1, cols=2, subplot_titles=("Distribución de precios", "Diagrama de caja de precios"))

# Histograma
hist = px.histogram(df, x="price", nbins=30, labels={"price": "Precio"})
for trace in hist.data:
    fig.add_trace(trace, row=1, col=1)

# Boxplot
box = px.box(df, y="price", labels={"price": "Precio"})
for trace in box.data:
    fig.add_trace(trace, row=1, col=2)

# Ajustes generales
fig.update_layout(
    width=900,
    height=500,
    showlegend=False
)

fig.show()

### ***Analisis de generos***

In [ ]:

# "Explota" cada género en filas individuales
df_exploded = df.explode('genres')

# Obtiene los dummies y agrupa por juego original
dummies = pd.get_dummies(df_exploded['genres'])

# Los suma por índice original y se los agrega al DataFrame original
df = df.join(dummies.groupby(df_exploded.index).sum())

# Crear tabla de recuento
juegos = pd.DataFrame({
    'Genero': dummies.columns,
    'Cantidad': dummies.sum().values
}).sort_values(by='Cantidad', ascending=False)



#### Cantidad total de juegos por genero

In [29]:
# Gráfico de barras de géneros más populares
fig = px.bar(
    juegos,
    x='Genero',
    y='Cantidad',
    title='<b>Popularidad de géneros de videojuegos</b>',
    labels={'Genero': 'Género', 'Cantidad': 'Número de juegos'},
    text='Cantidad',
    color_discrete_sequence=['#17becf'],
    width=900
)

# Ordenar de mayor a menor
fig.update_layout(xaxis={'categoryorder': 'total descending'})

fig.show()

#### Top 5 de generos mas jugados

In [30]:
# Agrupar y obtener top 5
juegos_top5 = (
    juegos.groupby('Genero')
    .sum(numeric_only=True)['Cantidad']
    .sort_values(ascending=False)
    .head(5)
    .reset_index()
)

# Gráfico de barras
fig = px.bar(
    juegos_top5,
    x='Genero',
    y='Cantidad',
    title='<b>Top 5 géneros de videojuegos</b>',
    labels={'Genero': 'Género', 'Cantidad': 'Número de juegos'},
    text='Cantidad',
    color_discrete_sequence=['#17becf'],
    width=800
)

# Ordenar de mayor a menor
fig.update_layout(xaxis={'categoryorder': 'total descending'})

fig.show()

### ***Cantidad de juegos lanzados por año***

In [31]:
# Contar juegos por año
juegosPorAño = df['release_date'].value_counts().sort_index()
juegosPorAño = juegosPorAño.reset_index()
juegosPorAño.columns = ['Año', 'Cantidad']

# Gráfico de barras
fig = px.bar(
    juegosPorAño,
    x='Año',
    y='Cantidad',
    title='<b>Cantidad de juegos lanzados por año</b>',
    labels={'Año': 'Año de lanzamiento', 'Cantidad': 'Cantidad de juegos'},
    text='Cantidad',
    color_discrete_sequence=['#17becf'],
    width=900
)

fig.update_layout(xaxis=dict(tickangle=90))  # Rotar etiquetas del eje X
fig.show()

### ***Analisis de recommend***

#### Cantidad de recomendaciones

In [33]:
# Contar recomendaciones y renombrar índices
recomendaciones = df['recommend'].value_counts().reset_index()
recomendaciones.columns = ['Recomendacion', 'Cantidad']
recomendaciones['Recomendacion'] = recomendaciones['Recomendacion'].map({1: 'Positivas', 0: 'Negativas'})


# Gráfico de barras horizontales
fig = px.bar(
    recomendaciones,
    x='Cantidad',
    y='Recomendacion',
    orientation='h',
    title='<b>Cantidad de recomendaciones</b>',
    labels={'Cantidad': 'Cantidad', 'Recomendacion': 'Recomendación'},
    text='Cantidad',
    color_discrete_sequence=['#17becf'],
    width=700
)

fig.show()

#### Top 5 de juegos mas recomendados

In [34]:
# Agrupar y obtener el top 5
juegosRecomend = (
    df[['item_name', 'recommend']]
    .groupby('item_name', as_index=False)['recommend']
    .sum()
    .sort_values(by='recommend', ascending=False)
    .head(5)
)

# Gráfico de barras
fig = px.bar(
    juegosRecomend,
    x='item_name',
    y='recommend',
    title='<b>Top 5 juegos más recomendados</b>',
    labels={'item_name': 'Juego', 'recommend': 'Cantidad de recomendaciones'},
    text='recommend',
    color_discrete_sequence=['#17becf'],
    width=800
)

# Rotar etiquetas en eje X
fig.update_layout(xaxis_tickangle=60)

fig.show()

### ***Analisis de sentiment_analysis***

#### Cantidad total de sentiment_analysis

In [35]:
# Contar valores y mapear etiquetas
sentimientos = df['sentiment_analysis'].value_counts().reset_index()
sentimientos.columns = ['sentiment_analysis', 'Cantidad']
sentimientos['sentiment_analysis'] = sentimientos['sentiment_analysis'].map({
    0: 'Positivas',
    1: 'Neutras',
    2: 'Negativas'
})

# Gráfico de barras
fig = px.bar(
    sentimientos,
    x='sentiment_analysis',
    y='Cantidad',
    title='<b>Cantidad de sentiment_analysis</b>',
    labels={'sentiment_analysis': 'Sentimiento', 'Cantidad': 'Cantidad'},
    text='Cantidad',
    color_discrete_sequence=['#17becf'],
    width=700
)

# Ordenar según 0,1,2 (Positivas, Neutras, Negativas)
orden = ['Positivas', 'Neutras', 'Negativas']
fig.update_xaxes(categoryorder='array', categoryarray=orden)

fig.show()

#### Top 5 de juegos con mejor reviws segun segun sentiment_analysis

In [36]:
# Agrupar, sumar y obtener top 5
juegosAnalysis = (
    df[['item_name', 'sentiment_analysis']]
    .groupby('item_name', as_index=False)['sentiment_analysis']
    .sum()
    .sort_values(by='sentiment_analysis', ascending=False)
    .head(5)
)

# Gráfico de barras
fig = px.bar(
    juegosAnalysis,
    x='item_name',
    y='sentiment_analysis',
    title='<b>Top 5 juegos con mayor cantidad de sentiment_analysis</b>',
    labels={'item_name': 'Juego', 'sentiment_analysis': 'Cantidad de sentimientos'},
    text='sentiment_analysis',
    color_discrete_sequence=['#17becf'],
    width=800
)

# Rotar etiquetas del eje X
fig.update_layout(xaxis_tickangle=60)

fig.show()